### URL parsing

In [1]:
import urllib.parse

# 한글 문자열
text = "강남역 맛집"

# URL 인코딩
encoded_text = urllib.parse.quote(text)

base_path=f"https://www.google.co.kr/maps/search/{encoded_text}"

print(encoded_text)

%EA%B0%95%EB%82%A8%EC%97%AD%20%EB%A7%9B%EC%A7%91


### googlemap_info

In [2]:
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.expected_conditions import visibility_of_all_elements_located,presence_of_element_located,presence_of_all_elements_located

# 레스토랑 db insert 기초 틀
def restuarant_properties(search_term,restaurant_name,url,address,rating,category,**kwrargs):
    
    properties={"검색어" : {"rich_text" : [{"text": {"content": search_term}}]},
                "음식점이름" : {"title" : [{"text" : {'content': restaurant_name}}]},
                "URL" : {"url" : url},
                "주소" : {"rich_text" : [{"text": {"content": address}}]},
                "별점" : {"number" : rating},
                "카테고리" : {"rich_text" : [{"text": {"content": category}}]}
                }
    return properties

# 리뷰 db insert 기초 틀
def review_properties(restaurant_name,user,review,**kwargs):

    properties={"음식점이름" : {"title" : [{"text": {"content": restaurant_name}}]},
                "유저" : {"rich_text" : [{"text": {"content": user}}]},
                "리뷰" : {"rich_text" : [{"text": {"content": review}}]}
                }
    return properties


info_num=20

Main_Drive=Chrome()
Main_Drive.get(base_path)
Sub_Drive=Chrome()

main_box=WebDriverWait(Main_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="feed"]']))

while True:
    main_box.send_keys(Keys.PAGE_DOWN)

    feed_box=WebDriverWait(Main_Drive,3).until(
        visibility_of_all_elements_located([By.CLASS_NAME,"hfpxzc"]))
    
    if len(feed_box)>info_num:
        break

infos=[]

for sub_feed in feed_box:
    search_term=text #위의 코드 에서 가져옴 수정필요
    restaurant_name=sub_feed.get_attribute("aria-label")
    url=sub_feed.get_attribute("href")

    infos.append(
        restuarant_properties(search_term,restaurant_name,url,None,None,None)
    )

for sub_info in infos:
    Sub_Drive.get(sub_info["URL"])

    #별점
    rating=WebDriverWait(Sub_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="img"]'])).get_attribute("aria-label")
    # 전처리해야함
    rating="".join([term for term in rating if not term.isalpha()]).strip()
    rating=float(rating)

    #음식점 카테고리
    category=WebDriverWait(Sub_Drive,3).until(
    presence_of_element_located([By.CSS_SELECTOR,'button.DkEaL'])).text

    #정보선택칸
    tabs=WebDriverWait(Sub_Drive,3).until(
    presence_of_all_elements_located([By.XPATH, '//*[@role="tab"]']))
    tab_select={element.text:element for element in tabs}
    
    #하위 정보칸
    info_region=WebDriverWait(Sub_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="region"]']))
    #주소
    address=WebDriverWait(info_region,3).until(
    presence_of_element_located([By.XPATH, '//*[@data-item-id="address"]']))
    

/Users/ijeonghun/Documents/github/GoogleMap/googlemap/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [105]:
Main_Drive=Chrome()
Main_Drive.get(base_path)
info_num=30
#Sub_Drive=Chrome()

main_box=WebDriverWait(Main_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="feed"]']))

while True:
    main_box.send_keys(Keys.PAGE_DOWN)

    feed_box=WebDriverWait(Main_Drive,3).until(
        visibility_of_all_elements_located([By.CSS_SELECTOR,"a.hfpxzc"]))
    
    if len(feed_box)>info_num:
        break

infos=[]

for sub_feed in feed_box:
    search_term=text #위의 코드 에서 가져옴 수정필요
    restaurant_name=sub_feed.get_attribute("aria-label")
    url=sub_feed.get_attribute("href")

    infos.append(
        restuarant_properties(search_term,restaurant_name,url,None,None,None)
    )

In [107]:
Main_Drive.quit()

In [122]:
#%%writefile GooglemapCraw.py
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import urllib.parse
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.expected_conditions import visibility_of_all_elements_located,presence_of_element_located,presence_of_all_elements_located


class GooglemapCraw():
    def  __init__(self,search_term,craw_num:int,optional:bool=True):

        # Chrome 옵션 설정
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # Headless 모드

        if optional:
            self.main_drive=Chrome(options=chrome_options) # url까지 크롤
            self.sub_drive=Chrome(options=chrome_options) # 기본정보 크롤
        else:
            self.main_drive=Chrome()
            self.sub_drive=Chrome()
        self.craw_num=craw_num
        self.search_term=search_term
        self.infos=[] # 임시 저장공간

        encoded_text = urllib.parse.quote(self.search_term)
        self.base_path=f"https://www.google.co.kr/maps/search/{encoded_text}" # 크롤링이 될 주소(파싱될 예정)
        pass

    def __restuarant_properties(self,search_term,restaurant_name,url,address,rating,category,**kwrargs):
    
        properties={"검색어" : {"rich_text" : [{"text": {"content": search_term}}]},
                "음식점이름" : {"title" : [{"text" : {'content': restaurant_name}}]},
                "URL" : {"url" : url},
                "주소" : {"rich_text" : [{"text": {"content": address}}]},
                "별점" : {"number" : rating},
                "카테고리" : {"rich_text" : [{"text": {"content": category}}]}
                }
        return properties

    # 리뷰 db insert 기초 틀
    def __review_properties(self,restaurant_name,user,review,**kwargs):

        properties={"음식점이름" : {"title" : [{"text": {"content": restaurant_name}}]},
                    "유저" : {"rich_text" : [{"text": {"content": user}}]},
                    "리뷰" : {"rich_text" : [{"text": {"content": review}}]}
                    }
        return properties
    
    def craw(self):
        self.main_drive.get(self.base_path)
        # 스크롤을 내리는 공간
        main_box=WebDriverWait(self.main_drive,3).until(
        presence_of_element_located([By.XPATH, '//*[@role="feed"]']))

        while True:
            main_box.send_keys(Keys.PAGE_DOWN)

            feed_box=WebDriverWait(self.main_drive,3).until(
                visibility_of_all_elements_located([By.CSS_SELECTOR,"a.hfpxzc"]))
            
            if len(feed_box)>self.craw_num:
                break
        
        for sub_feed in feed_box:
            #음식점 이름
            restaurant_name=sub_feed.get_attribute("aria-label")
            url=sub_feed.get_attribute("href")

            self.infos.append(
                {"search_term":self.search_term,
                 "restaurant_name":restaurant_name,
                 "url":url}
            )

        for sub_info in self.infos:
            self.sub_drive.get(sub_info["url"])

            #별점
            rating=WebDriverWait(self.sub_drive,3).until(
            presence_of_element_located([By.XPATH, '//*[@role="img"]'])).get_attribute("aria-label")
            #전처리
            rating="".join([term for term in rating if not term.isalpha()]).strip()
            rating=float(rating)

            #음식점 카테고리
            category=WebDriverWait(self.sub_drive,3).until(
            presence_of_element_located([By.CSS_SELECTOR,'button.DkEaL'])).text

            #정보선택칸
            tabs=WebDriverWait(self.sub_drive,3).until(
            presence_of_all_elements_located([By.XPATH, '//*[@role="tab"]']))
            tab_select={element.text:element for element in tabs} # 이후 사용할 예정
            
            #하위 정보칸
            info_region=WebDriverWait(self.sub_drive,3).until(
            presence_of_element_located([By.XPATH, '//*[@role="region"]']))
            #주소
            address=WebDriverWait(info_region,3).until(
            presence_of_element_located([By.XPATH, '//*[@data-item-id="address"]'])).text

            sub_info.update(
                {"address":address,
                 "rating":rating,
                 "category":category}
            )
    def close(self):
        self.main_drive.quit()
        self.sub_drive.quit()

In [123]:
test_class=GooglemapCraw("강남역 맛집",20,False)

In [124]:
test_class.craw()

In [125]:
test_class.infos

[{'search_term': '강남역 맛집',
  'restaurant_name': '바비레드 강남본점',
  'url': 'https://www.google.co.kr/maps/place/%EB%B0%94%EB%B9%84%EB%A0%88%EB%93%9C+%EA%B0%95%EB%82%A8%EB%B3%B8%EC%A0%90/data=!4m7!3m6!1s0x357ca3e29805ba2f:0x339ec75093d60a3b!8m2!3d37.5026295!4d127.0278967!16s%2Fg%2F11bbtlp9qn!19sChIJL7oFmOKjfDUROwrWk1DHnjM?authuser=0&hl=ko&rclk=1',
  'address': <selenium.webdriver.remote.webelement.WebElement (session="0454a257120a46167db9f431ea981f6f", element="f.CAB1FB6440318A6C04C7F291C6C68E7F.d.022BBE45EE35900D1DB910B5B755AD1B.e.339")>,
  'rating': 4.3,
  'category': '퓨전 음식점'},
 {'search_term': '강남역 맛집',
  'restaurant_name': '정돈 강남점',
  'url': 'https://www.google.co.kr/maps/place/%EC%A0%95%EB%8F%88+%EA%B0%95%EB%82%A8%EC%A0%90/data=!4m7!3m6!1s0x357ca39dc0941e5f:0x73d7fe7d16782344!8m2!3d37.5039525!4d127.026436!16s%2Fg%2F11gxs6965c!19sChIJXx6UwJ2jfDURRCN4Fn3-13M?authuser=0&hl=ko&rclk=1',
  'address': <selenium.webdriver.remote.webelement.WebElement (session="0454a257120a46167db9f431ea981f6f"

In [126]:
test_class.close()

In [8]:
test_driver=Chrome()
test_driver.get(url)

In [49]:
test_driver.find_element(By.CSS_SELECTOR,"button.DkEaL").text

'닭요리전문점'

In [28]:
get_rating=test_driver.find_element(By.CLASS_NAME,"ceNzKf").get_attribute("aria-label")

In [16]:
take_img=WebDriverWait(test_driver,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="img"]']))

In [41]:
res_info=WebDriverWait(test_driver,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="region"]']))

In [ ]:
address=WebDriverWait(res_info,3).until(
    presence_of_element_located([By.XPATH, '//*[@data-item-id="address"]']))

In [45]:
address.get_attribute("aria-label")

'주소: 서울특별시 강남구 강남대로96길 9 '

In [50]:
tabs=WebDriverWait(test_driver,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="tab"]']))

In [52]:
tabs.

'개요'

In [53]:
test_driver.quit()

In [3]:
Main_Drive.quit()

### Restaurant_info

In [88]:
feed_box[0].get_attribute("aria-label")

'바비레드 강남본점'

In [78]:
main_box.get_attribute("role")

'feed'

In [72]:
feed_box[0].get_attribute("aria-label")

'바비레드 강남본점'

In [89]:
Main_Drive.quit()

In [59]:
test_elements=Main_Drive.find_elements(By.CLASS_NAME,"hfpxzc")

MaxRetryError: HTTPConnectionPool(host='localhost', port=57278): Max retries exceeded with url: /session/099672721e87bc05898169d8f5370121/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109ffabb0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [47]:
test_elements[-1].get_attribute("aria-label")

'땀땀'

In [48]:
test_elements[-1].get_attribute("href")

'https://www.google.co.kr/maps/place/%EB%95%80%EB%95%80/data=!4m7!3m6!1s0x357cbba6cab8be71:0x123ebf3924dc0759!8m2!3d37.5003867!4d127.027994!16s%2Fg%2F11hcqrdf1p!19sChIJcb64yqa7fDURWQfcJDm_PhI?authuser=0&hl=ko&rclk=1'

In [49]:
Main_Drive.quit()

In [22]:
# 검색창
feedbox=Main_Drive.find_elements(By.TAG_NAME,"div")

In [25]:
feed_test=[element for element in feedbox if element.get_attribute("role")=="feed"]

In [29]:
feed_test[0].get_attribute("aria-label")

'강남역 맛집에 대한 검색결과'

In [33]:
from selenium.webdriver.common.keys import Keys

for _ in range(10):
    feed_test[0].send_keys(Keys.PAGE_DOWN)

테스트 끝.

In [37]:
Main_Drive.quit()

In [40]:
from threading import Thread

구조 

step 1 : url get

step 2 : url을 통해 세부정보 get